In [1]:
import urllib3
import json
import pandas as pd
import numpy  as np
import time
import datetime

In [2]:
# 検索クエリ
def req_params(start, end):
    params = {'query'        : '(btc OR bitcoin) -giveaway lang:en -is:retweet -is:reply -is:nullcast',
              'start_time'   : '2021-' + start + '-01T00:00:00Z',
              'end_time'     : '2021-' + end   + '-01T00:00:00Z',
              'max_results'  : 500,
              'expansions'   : 'author_id',
              'tweet.fields' : 'text,created_at,public_metrics',
              'user.fields'  : 'name,username,public_metrics'
              }
    
    return params

In [3]:
# http,key
config = pd.read_csv("./config_academic.csv")
twitterAPIBearerToken=config['twitterApiBearerToken'][0]

http = urllib3.PoolManager()
key  = twitterAPIBearerToken

In [4]:
# ツイート収集リクエスト
def req_tweet(http, key, searchFeild):
    url  = 'https://api.twitter.com/2/tweets/search/all'

    req = http.request('GET',
                        url,
                        headers= {'Authorization': 'Bearer '+key},
                        fields = searchFeild
                      )

    result = json.loads(req.data)
    if (req.status != 200):
        print('検索失敗')
        print(req.status)
        print(result['errors'])
        
    return result

In [5]:
#ツイート収集実行
def get_tweet(params):
    next_token    = ''
    break_flag    = False
    request_count = 0
    tweet_count   = 0
    counter       = 1
    t_data        = []  # tweetのリスト
    u_data        = []  # userのリスト

    # 次ページがなくなるまで次ページのクエリを取得
    while True:
        try:
            data['meta']['next_token']

        # n+1回目：次ページがない(next_tokenがない)のでループを抜ける   
        except KeyError:
            break_flag = True

        # 1回目 
        except NameError:
            query = params

        ## 2≦k≦n回目　
        else:
            next_token = data['meta']['next_token']
            params['next_token'] = next_token
            query = params

        finally:
            if break_flag == True:
                break

            # 1リクエストで収集したデータ
            data = req_tweet(http, key, query)

            # tweet.fields
            if ('data' in data):          
                for i in range(data['meta']['result_count']):
                    t_data.append([data['data'][i]['text'],
                                   data['data'][i]['public_metrics']['like_count'],
                                   data['data'][i]['public_metrics']['retweet_count'],
                                   data['data'][i]['created_at'],
                                   data['data'][i]['author_id']])
                    i += 1

            # user.fields
            if ('includes' in data):
                for user in data['includes']['users']:
                    u_data.append([user['name'],
                                   '@'+user['username'],
                                   user['public_metrics']['following_count'],
                                   user['public_metrics']['followers_count'],
                                   user['id']])

            tweet_count += data['meta']['result_count']
            request_count += 1
            if request_count >= 300: # 300requestを超えたら止める
                dt_now = datetime.datetime.now()
                print(dt_now)
                print('{}回目. 300リクエストを超えるため、15分間停止します...'.format(counter))
                time.sleep(15.01*60) # 15分間（余裕をみてプラス1秒弱）中断
                dt_now = datetime.datetime.now()
                print(dt_now)
                print('15分経過、収集を再開します')
                counter += 1
                request_count = 0
                
    return t_data,u_data,tweet_count

In [6]:
# csv出力
def to_csv(i,start_time, end_time, t_data, u_data, tweet_count):
    print("2021年{}月のツイート{}件収集完了".format(i,tweet_count))

    # 収集ログ
    dt_now = datetime.datetime.now()
    df_log = pd.read_csv('./tweet-of-btc/assembling-log.csv')
    s = pd.Series([dt_now, start_time, end_time, tweet_count], index=['収集時刻', '区間(start)','区間(end)','収集件数'])
    df_log = df_log.append(s, ignore_index=True)
    df_log.to_csv('./tweet-of-btc/assembling-log.csv',index=False)

    # t_data, u_data
    df1 = pd.DataFrame(data=t_data, columns=['Tweet','Favorite','RT','created_at','author_id'])
    df1.to_csv(f'./tweet-of-btc/2021-0{i}_tlist.csv', index=False)
    df2 = pd.DataFrame(data=u_data, columns=['Name','User_Name','Following','Followers','author_id'])
    df2.to_csv(f'./tweet-of-btc/2021-0{i}_ulist.csv', index=False)

    # t_data × u_data
#    df3 = pd.merge(df2 ,df1, on='author_id')

    # ツイート時刻順にソート
#   df3['created_at'] = pd.to_datetime(df3['created_at'], infer_datetime_format= True)
#   sorted_df = df3.sort_values(by = 'created_at', ascending = True, inplace = True) 
#   df3.to_csv('./tweet-of-btc/2021-0{i}.csv', index=False)

In [7]:
# main
dt_now = datetime.datetime.now()
print(dt_now)
print('収集を開始します')
start = '0' + str(3)
end   = '0' + str(4)
params = req_params(start, end)
t_data, u_data, tweet_count = get_tweet(params)
to_csv(3,params['start_time'],params['end_time'],t_data, u_data, tweet_count)

2021-09-05 12:01:30.064489
1回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 12:16:30.666534
15分経過、収集を再開します
2021-09-05 12:23:17.148798
2回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 12:38:17.760486
15分経過、収集を再開します
2021-09-05 12:44:57.001764
3回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 12:59:57.604496
15分経過、収集を再開します
2021-09-05 13:06:35.012889
4回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 13:21:35.621555
15分経過、収集を再開します
2021-09-05 13:28:10.300385
5回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 13:43:10.909575
15分経過、収集を再開します
2021-09-05 13:49:39.948674
6回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 14:04:40.558787
15分経過、収集を再開します
2021-09-05 14:11:06.901513
7回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 14:26:07.513392
15分経過、収集を再開します
2021-09-05 14:32:23.059910
8回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 14:47:23.671407
15分経過、収集を再開します
2021-09-05 14:53:40.036463
9回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 15:08:40.642279
15分経過、収集を再開します
2021-09-05 15:14:50.026133
10回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 15:29:48.9

In [7]:
# main
start = '0' + str(4)
end   = '0' + str(5)
params = req_params(start, end)
t_data, u_data, tweet_count = get_tweet(params)
to_csv(4,params['start_time'],params['end_time'],t_data, u_data, tweet_count)

2021-09-05 16:52:07.868561
1回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 17:07:08.480172
15分経過、収集を再開します
2021-09-05 17:14:52.718423
2回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 17:29:53.323189
15分経過、収集を再開します
2021-09-05 17:37:07.195218
3回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 17:52:07.809163
15分経過、収集を再開します
2021-09-05 17:59:08.818279
4回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 18:14:09.420680
15分経過、収集を再開します
2021-09-05 18:21:01.793946
5回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 18:36:02.398826
15分経過、収集を再開します
2021-09-05 18:42:49.978707
6回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 18:57:50.588498
15分経過、収集を再開します
2021-09-05 19:04:25.496983
7回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 19:19:26.098448
15分経過、収集を再開します
2021-09-05 19:25:54.174117
8回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 19:40:54.784196
15分経過、収集を再開します
2021-09-05 19:47:36.772667
9回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 20:02:37.377083
15分経過、収集を再開します
2021-09-05 20:09:32.632580
10回目. 300リクエストを超えるため、15分間停止します...
2021-09-05 20:24:33.2

In [8]:
# main
dt_now = datetime.datetime.now()
print(dt_now)
print('収集を開始します')
start = '0' + str(5)
end   = '0' + str(6)
params = req_params(start, end)
t_data, u_data, tweet_count = get_tweet(params)
to_csv(5,params['start_time'],params['end_time'],t_data, u_data, tweet_count)

2021-09-06 12:26:01.039620
収集を開始します
2021-09-06 12:33:02.235166
1回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 12:48:02.835588
15分経過、収集を再開します
2021-09-06 12:54:57.356708
2回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 13:09:57.963674
15分経過、収集を再開します
2021-09-06 13:16:46.828870
3回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 13:31:47.429124
15分経過、収集を再開します
2021-09-06 13:38:26.794221
4回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 13:53:24.952957
15分経過、収集を再開します
2021-09-06 14:00:08.080950
5回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 14:15:08.690609
15分経過、収集を再開します
2021-09-06 14:26:52.679619
6回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 14:41:53.287609
15分経過、収集を再開します
2021-09-06 14:51:55.592540
7回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 15:06:56.207907
15分経過、収集を再開します
2021-09-06 15:18:11.670962
8回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 15:33:12.275228
15分経過、収集を再開します
2021-09-06 15:44:37.258680
9回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 15:59:37.861567
15分経過、収集を再開します
2021-09-06 16:08:33.392959
10回目. 300リクエストを超えるた

In [7]:
# main
dt_now = datetime.datetime.now()
print(dt_now)
print('収集を開始します')
start = '0' + str(6)
end   = '0' + str(7)
params = req_params(start, end)
t_data, u_data, tweet_count = get_tweet(params)
to_csv(6,params['start_time'],params['end_time'],t_data, u_data, tweet_count)

2021-09-06 20:36:45.689930
収集を開始します
2021-09-06 20:47:23.761776
1回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 21:02:24.373417
15分経過、収集を再開します
2021-09-06 21:11:08.906694
2回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 21:26:09.521026
15分経過、収集を再開します
2021-09-06 21:34:22.446751
3回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 21:49:23.054413
15分経過、収集を再開します
2021-09-06 21:57:03.399403
4回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 22:12:04.003513
15分経過、収集を再開します
2021-09-06 22:19:28.909992
5回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 22:34:29.512284
15分経過、収集を再開します
2021-09-06 22:41:51.613682
6回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 22:56:52.227702
15分経過、収集を再開します
2021-09-06 23:04:04.251149
7回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 23:19:04.857119
15分経過、収集を再開します
2021-09-06 23:26:09.545675
8回目. 300リクエストを超えるため、15分間停止します...
2021-09-06 23:41:10.148815
15分経過、収集を再開します
2021-09-06 23:47:58.352166
9回目. 300リクエストを超えるため、15分間停止します...
2021-09-07 00:02:58.965213
15分経過、収集を再開します
2021-09-07 00:09:38.308987
10回目. 300リクエストを超えるた